In [11]:
import pandas as pd
import os

dpath = os.path.expanduser('~/Dropbox/Projects/data/')

In [46]:
diag = pd.read_csv(os.path.expanduser(dpath + 'acute_inflammations/diagnosis.data'), 
                   header=None, sep='\t', skip_blank_lines=True, 
                  true_values=['yes'], false_values=['no'])
diag.columns = ['tmp', 'nausea', 'lumbar_pain', 'urine_pushing', 'micturition_pain', 'burning', 'inflammation', 'nephritis']
diag.head()

,tmp,nausea,lumbar_pain,urine_pushing,micturition_pain,burning,inflammation,nephritis
0,"35,5",False,True,False,False,False,False,False
1,"35,9",False,False,True,True,True,True,False
2,"35,9",False,True,False,False,False,False,False
3,"36,0",False,False,True,True,True,True,False
4,"36,0",False,True,False,False,False,False,False


In [ ]:
from sklearn.linear_model import LogisticRegression

splits     = cv.train_test_split(iris.data, iris.target, test_size=0.2)
X_train, X_test, y_train, y_test = splits

model      = LogisticRegression()
model.fit(X_train, y_train)

expected   = y_test
predicted  = model.predict(X_test)

print classification_report(expected, predicted)

In [25]:
import pyspark
import pyspark.sql.functions as F


In [55]:
diags = spark.createDataFrame(diag)
diags = diags.withColumn('tmp', F.regexp_replace(diags.tmp, ',', '.').cast('float'))\
    .withColumn('nausea', diags.nausea.cast('int'))\
    .withColumn('lumbar_pain', diags.lumbar_pain.cast('int'))\
    .withColumn('urine_pushing', diags.urine_pushing.cast('int'))\
    .withColumn('micturition_pain', diags.micturition_pain.cast('int'))\
    .withColumn('burning', diags.burning.cast('int'))\
    .withColumn('inflammation', diags.inflammation.cast('int'))\
    .withColumn('nephritis', diags.nephritis.cast('int'))

print diags.show(2)
print diags.printSchema()
train, test = diags.randomSplit([0.7, 0.3])

+----+------+-----------+-------------+----------------+-------+------------+---------+
| tmp|nausea|lumbar_pain|urine_pushing|micturition_pain|burning|inflammation|nephritis|
+----+------+-----------+-------------+----------------+-------+------------+---------+
|35.5|     0|          1|            0|               0|      0|           0|        0|
|35.9|     0|          0|            1|               1|      1|           1|        0|
+----+------+-----------+-------------+----------------+-------+------------+---------+
only showing top 2 rows

None
root
 |-- tmp: float (nullable = true)
 |-- nausea: integer (nullable = true)
 |-- lumbar_pain: integer (nullable = true)
 |-- urine_pushing: integer (nullable = true)
 |-- micturition_pain: integer (nullable = true)
 |-- burning: integer (nullable = true)
 |-- inflammation: integer (nullable = true)
 |-- nephritis: integer (nullable = true)

None


In [59]:
from pyspark.ml.feature import VectorAssembler, StringIndexer

featuresCols = diags.columns
featuresCols.remove('inflammation')
featuresCols.remove('nephritis')
# featuresCols = 'lumbar_pain'
# This concatenates all feature columns into a single feature vector in a new column "rawFeatures".
# indexer = StringIndexer(inputCol=featuresCols, outputCol='tmp2')
# train2 = indexer.fit(train).transform(train)
# train2.show()
train3 = VectorAssembler(inputCols=featuresCols, outputCol="features").transform(train)
train3.show(3, truncate=False)

+----+------+-----------+-------------+----------------+-------+------------+---------+----------------------------------------+
|tmp |nausea|lumbar_pain|urine_pushing|micturition_pain|burning|inflammation|nephritis|features                                |
+----+------+-----------+-------------+----------------+-------+------------+---------+----------------------------------------+
|35.5|0     |1          |0            |0               |0      |0           |0        |(6,[0,2],[35.5,1.0])                    |
|35.9|0     |0          |1            |1               |1      |1           |0        |[35.900001525878906,0.0,0.0,1.0,1.0,1.0]|
|35.9|0     |1          |0            |0               |0      |0           |0        |(6,[0,2],[35.900001525878906,1.0])      |
+----+------+-----------+-------------+----------------+-------+------------+---------+----------------------------------------+
only showing top 3 rows



In [60]:
# from pyspark.ml.regression import LinearRegression
from pyspark.ml.classification import LogisticRegression

# lr = LinearRegression(labelCol="hba1c", maxIter=100, regParam=0.3, elasticNetParam=0.8)
# lr = LinearRegression(labelCol="hba1c")
lr = LogisticRegression(labelCol="inflammation", maxIter=10, regParam=0.01)

# Fit the model
lrModel = lr.fit(train3)

# Print the coefficients and intercept for linear regression
print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))

Coefficients: [-0.0735675831176,1.64218712081,-3.94462071627,4.93030250586,3.18539011674,-1.00491962156]
Intercept: 0.284071149325
